<a href="https://colab.research.google.com/github/jeffersonramelo/Coleta-dados-SEC/blob/main/dados_financeiros_SEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar módulos
import requests
import pandas as pd

# Cabeçalho para a requisição
headers = {'User-Agent': "jeffersonramelo@ufpi.edu.br"}

# Obter dados de todas as empresas listadas
response = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)
companies = pd.DataFrame.from_dict(response.json(), orient='index')
companies['cik_str'] = companies['cik_str'].astype(str).str.zfill(10)

# Inicializar DataFrame para armazenar os resultados
results = pd.DataFrame(columns=['cik', 'company_name', 'fiscal_year_end', 'net_income'])

# Iterar sobre cada empresa
for index, row in companies.iterrows():
    cik = row['cik_str']
    company_name = row['title']

    try:
        # Obter metadados de arquivamento específicos da empresa
        facts_url = f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json'
        facts_response = requests.get(facts_url, headers=headers)

        if facts_response.status_code == 200:
            facts_data = facts_response.json()

            if 'us-gaap' in facts_data['facts'] and 'NetIncomeLoss' in facts_data['facts']['us-gaap']:
                net_income_data = facts_data['facts']['us-gaap']['NetIncomeLoss']

                if 'USD' in net_income_data['units']:
                    # Filtrar para incluir apenas dados de 2023
                    year_2023_data = [data for data in net_income_data['units']['USD'] if '2023' in data['end']]

                    if year_2023_data:
                        latest_year_data = max(year_2023_data, key=lambda x: x['end'])

                        # Criar um DataFrame temporário para o resultado atual
                        temp_df = pd.DataFrame([{
                            'cik': cik,
                            'company_name': company_name,
                            'fiscal_year_end': latest_year_data['end'],
                            'net_income': latest_year_data['val']
                        }])

                        # Concatenar o DataFrame temporário com o DataFrame de resultados
                        results = pd.concat([results, temp_df], ignore_index=True)
                    else:
                        # Adicionar linha com valores nulos se não houver dados de 2023
                        results = pd.concat([results, pd.DataFrame([{
                            'cik': cik,
                            'company_name': company_name,
                            'fiscal_year_end': None,
                            'net_income': None
                        }], index=[index])], ignore_index=True)
                else:
                    # Adicionar linha com valores nulos se não houver dados em USD
                    results = pd.concat([results, pd.DataFrame([{
                        'cik': cik,
                        'company_name': company_name,
                        'fiscal_year_end': None,
                        'net_income': None
                    }], index=[index])], ignore_index=True)
            else:
                # Adicionar linha com valores nulos se não houver dados de lucro líquido
                results = pd.concat([results, pd.DataFrame([{
                    'cik': cik,
                    'company_name': company_name,
                    'fiscal_year_end': None,
                    'net_income': None
                }], index=[index])], ignore_index=True)
        else:
            # Adicionar linha com valores nulos se não for possível acessar os dados
            results = pd.concat([results, pd.DataFrame([{
                'cik': cik,
                'company_name': company_name,
                'fiscal_year_end': None,
                'net_income': None
            }], index=[index])], ignore_index=True)

    except Exception as e:
        # Silenciosamente ignorar erros e adicionar linha com valores nulos
        results = pd.concat([results, pd.DataFrame([{
            'cik': cik,
            'company_name': company_name,
            'fiscal_year_end': None,
            'net_income': None
        }], index=[index])], ignore_index=True)

# Salvando os resultados
results.to_csv('net_income_2023.csv', index=False)


In [ ]:
import pandas as pd

# Caminho do arquivo CSV
caminho_do_arquivo = '/content/net_income_2023.csv'

# Ler o arquivo CSV
dados = pd.read_csv(caminho_do_arquivo)

# Exibir as primeiras linhas do DataFrame
dados


,cik,company_name,fiscal_year_end,net_income
0,789019,MICROSOFT CORP,2023-12-31,4.416100e+10
1,320193,Apple Inc.,2023-12-30,3.391600e+10
2,1045810,NVIDIA CORP,2023-10-29,1.747500e+10
3,1652044,Alphabet Inc.,2023-12-31,7.379500e+10
4,1018724,AMAZON COM INC,2023-12-31,3.042500e+10
...,...,...,...,...
10347,1748680,1WS Credit Income Fund,NaN,NaN
10348,1957489,Able View Global Inc.,2023-12-31,9.750046e+06
10349,1933644,"Medlive Technology Co., Ltd./ADR",NaN,NaN
10350,1062750,SAIPEM S P A /FI,NaN,NaN
